# Introdução

# Dependências

In [43]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install requests

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
import pandas.api.types as ptypes
import numpy as np
import requests
import tweepy

import json
import os

# Coleta

## Previsões

In [104]:
predictions_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
filename = 'image_predictions.tsv'
if not os.path.exists(filename):
    response = requests.get(predictions_url)
    with open(filename, 'wb') as file:
        file.write(response.content)

In [105]:
df_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
df_predictions_clean = df_predictions.copy()

In [106]:
df_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


## Tweets

In [107]:
df_tweets = pd.read_csv('twitter-archive-enhanced.csv')
df_tweets_clean = df_tweets.copy()

In [108]:
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

## Contagem de retweets e favoritos

In [9]:
consumer_key = os.environ['TWITTER_CONSUMER_KEY']
consumer_secret = os.environ['TWITTER_CONSUMER_SECRET']
access_token = os.environ['TWITTER_ACCESS_TOKEN']
access_token_secret = os.environ['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

twitter_api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [23]:
# Função para mapear o resultado de cada chamada API para um resultado JSON
def get_retweet_counts(tweet_id):
    try:
        tweet_status = twitter_api.get_status(tweet_id, tweet_mode='extended')
        return tweet_status._json
    except tweepy.TweepError:
        pass

In [109]:
filename = 'tweet_json.txt'

if not os.path.exists(filename):
    tweet_counts = df_tweets_clean['tweet_id'].apply(lambda tweet_id: get_retweet_counts(tweet_id))
    with open(filename, 'w+') as handle:
        for tweet_count in tweet_counts:
            handle.write(json.dumps(tweet_count) + '\n')

tweet_data = []
with open(filename, 'r') as handle:
    for line in handle:
        tweet_count = json.loads(line)
        tweet_data.append(tweet_count)

In [110]:
filtered_tweet_data = list(filter(lambda r: r is not None, tweet_data))

In [111]:
df_tweet_counts = pd.DataFrame(filtered_tweet_data)
df_tweet_counts_clean = df_tweet_counts.copy()

In [112]:
df_tweet_counts_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 32 columns):
contributors                     0 non-null object
coordinates                      0 non-null object
created_at                       2340 non-null object
display_text_range               2340 non-null object
entities                         2340 non-null object
extended_entities                2065 non-null object
favorite_count                   2340 non-null int64
favorited                        2340 non-null bool
full_text                        2340 non-null object
geo                              0 non-null object
id                               2340 non-null int64
id_str                           2340 non-null object
in_reply_to_screen_name          78 non-null object
in_reply_to_status_id            78 non-null float64
in_reply_to_status_id_str        78 non-null object
in_reply_to_user_id              78 non-null float64
in_reply_to_user_id_str          78 non-null obj

# Avaliação

## Estrutura do DataFrame de previsões

In [113]:
df_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


## Estrutura do DataFrame de tweets

In [114]:
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

## Identificação de Retweets

In [115]:
df_retweets = df_tweets_clean[df_tweets_clean.in_reply_to_status_id.notnull()]
df_retweets.shape[0]

78

## Denominador com valor abaixo de 10

In [116]:
df_tweets_clean[df_tweets_clean.rating_denominator < 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
2335,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an Albanian 3 1/2 legged Episcopalian...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666287406...,1,2,an,None,None,None,None


## Denominador com valor acima de 10

In [121]:
df_tweets_clean[df_tweets_clean.rating_denominator > 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582...,4,20,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday here's 9 puppers on a bench. 99...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713900603...,99,90,None,None,None,None,None
1254,710658690886586372,NaN,NaN,2016-03-18 02:46:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a brigade of puppers. All look very pre...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/710658690...,80,80,None,None,None,None,None


## Tweets sem referência do nome do animal

In [117]:
df_no_name = df_tweets_clean[df_tweets_clean.name == 'None']
df_no_name.shape[0]

745

# Documentação dos Problemas de Qualidade

## Problemas de Qualidade

1. Tipo das colunas "in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id" deve ser alterado de float64 para int64 (seção "Estrutura do DataFrame de tweets")
2. Tipo das colunas "timestamp" e "retweeted_status_timestamp" deve ser alterado de string (object) para datetime
3. 78 tweets são retweets (seção "Identificação de Retweets")
4. 3 tweets com denominador abaixo de 10 (seção "Denominador com valor abaixo de 10")
5. 745 tweets não contém o nome do animal a qual se referem (seção "Tweets sem referência do nome do animal")

## Problemas de Arrumação

1. DataFrame com as previsões contem 9 colunas relacionadas aos valores de previsão, onde apenas 3 colunas devem ser utilizadas para normalizar estes valores (previsão, intervalo de confiança e valor booleano indicando se a previsão é uma raça de cachorro) (seção "Estrutura do DataFrame de previsões")
2. DataFrame com os tweets contem 4 colunas relacionadas ao "estágio" do animal, podendo ser normalizada para apenas uma coluna "estágio" (seção "Estrutura do DataFrame de tweets")

# Limpeza

## Ajustar tipo de dados das colunas "in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id" (float para int)

In [72]:
df_tweets_clean.in_reply_to_status_id = df_tweets_clean.in_reply_to_status_id.fillna(0).astype(int)
df_tweets_clean.in_reply_to_user_id = df_tweets_clean.in_reply_to_user_id.fillna(0).astype(int)
df_tweets_clean.retweeted_status_id = df_tweets_clean.retweeted_status_id.fillna(0).astype(int)
df_tweets_clean.retweeted_status_user_id = df_tweets_clean.retweeted_status_user_id.fillna(0).astype(int)
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null int64
in_reply_to_user_id           2356 non-null int64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null int64
retweeted_status_user_id      2356 non-null int64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: int64(7), object(10)
memor

### Teste

In [88]:
assert ptypes.is_integer_dtype(df_tweets_clean.in_reply_to_status_id)
assert ptypes.is_integer_dtype(df_tweets_clean.in_reply_to_user_id)
assert ptypes.is_integer_dtype(df_tweets_clean.retweeted_status_id)
assert ptypes.is_integer_dtype(df_tweets_clean.retweeted_status_user_id)

## Ajustar tipo de dados das colunas "timestamp" e "retweeted_status_timestamp" (string para datetime)

In [80]:
df_tweets_clean.timestamp = pd.to_datetime(df_tweets_clean.timestamp)
df_tweets_clean.retweeted_status_timestamp = pd.to_datetime(df_tweets_clean.retweeted_status_timestamp)
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null int64
in_reply_to_user_id           2356 non-null int64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null int64
retweeted_status_user_id      2356 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime64

### Teste

In [90]:
assert ptypes.is_datetime64_any_dtype(df_tweets_clean.timestamp)
assert ptypes.is_datetime64_any_dtype(df_tweets_clean.retweeted_status_timestamp)

## Descartar retweets

In [83]:
df_tweets_clean = df_tweets_clean[df_tweets_clean.in_reply_to_status_id == 0]
df_tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2278 non-null int64
in_reply_to_status_id         2278 non-null int64
in_reply_to_user_id           2278 non-null int64
timestamp                     2278 non-null datetime64[ns]
source                        2278 non-null object
text                          2278 non-null object
retweeted_status_id           2278 non-null int64
retweeted_status_user_id      2278 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2274 non-null object
rating_numerator              2278 non-null int64
rating_denominator            2278 non-null int64
name                          2278 non-null object
doggo                         2278 non-null object
floofer                       2278 non-null object
pupper                        2278 non-null object
puppo                         2278 non-null object
dtypes: datetime64

### Teste

In [91]:
assert len(df_tweets_clean[df_tweets_clean.in_reply_to_status_id > 0]) == 0

## Contagem de retweets e favoritos

In [62]:
df_tweet_counts_clean['retweet_count'] = df_tweet_counts_clean['retweet_count'].fillna(0).astype(int)
df_tweet_counts_clean['favorite_count'] = df_tweet_counts_clean['favorite_count'].fillna(0).astype(int)
df_tweet_counts_clean[(df_tweet_counts_clean.retweet_count.isnull()) | (df_tweet_counts_clean.favorite_count.isnull())]

,tweet_id,retweet_count,favorite_count


## Ajuste das predições

In [ ]:
df_predictions_1 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p1", "p1_conf", "p1_dog"]].copy()
df_predictions_2 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p2", "p2_conf", "p2_dog"]].copy()
df_predictions_3 = df_predictions_clean[["tweet_id", "jpg_url", "img_num", "p3", "p3_conf", "p3_dog"]].copy()

df_predictions_1.rename(columns={'p1': 'prediction', 'p1_conf': 'confidence_interval', 'p1_dog': 'is_dog_race'}, inplace=True)
df_predictions_2.rename(columns={'p2': 'prediction', 'p2_conf': 'confidence_interval', 'p2_dog': 'is_dog_race'}, inplace=True)
df_predictions_3.rename(columns={'p3': 'prediction', 'p3_conf': 'confidence_interval', 'p3_dog': 'is_dog_race'}, inplace=True)

df_predictions_clean = pd.concat([df_predictions_1, df_predictions_2, df_predictions_3])
df_predictions_clean.shape[0]